In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
today = datetime.now()
today = f"{str(today.year)[-2:]}{today.month:02}{today.day:02}"

os.makedirs(f'./code9/{today}', exist_ok=True)

In [48]:
import re
pkor = re.compile(r'[가-힣. ]+')
# peng = re.compile(r'[a-z ]+')

def keyword_search(args: list, word: str, sensitive='any') -> bool:
    isinList = [arg in word for arg in args if arg[0]!="~"]
    isnotinList = [arg[1:] in word if arg[0]=="~" else False for arg in args]
    if sensitive == 'all':
        return np.array(isinList).all() & (not np.array(isnotinList).any())
    elif sensitive == 'any':
        return np.array(isinList).any() & (not np.array(isnotinList).any())
    else:
        raise AssertionError("Unexpected condition. You select 'all' or 'any'.")

def goldendog(kor=None, eng=None, feecode=None, condition='and', inkor_cond='any', ineng_cond='any', df=None):
    ''' Dang dang E. Give some korean or englist text-list!
        If you give some word have in first letter "~", doggy will not retrieve that word.
            ex) "배고파 밥하고 김치먹자", "콩밥임 오늘 점심..."
                kor = ['밥', '김치'] --> Retrive "배고파 밥하고 김치먹자", "콩밥임 오늘 점심..."
                kor = ['밥', '~김치'] --> Retrive "콩밥임 오늘 점심..."
        '''
    assert isinstance(df, pd.DataFrame), "Give me a dataframe!"
    bird_kor = [e for e in df['kor'] if keyword_search(kor, word=pkor.findall(e)[0], sensitive=inkor_cond)] if kor != None else [""]
    bird_eng = [e for e in df['eng'] if keyword_search(eng, word=str(e).lower(), sensitive=ineng_cond)] if eng != None else [""]

    gold_ixs = []
    for i in range(len(df)):
        if feecode != None:
            if df.iloc[i, 0] in feecode:
                gold_ixs.append(df.iloc[[i], :].index[0])

        isin_kor = False if kor==None else df['kor'].iloc[i] in bird_kor
        isin_eng = False if eng==None else df['eng'].iloc[i] in bird_eng if pd.notnull(df['eng'].iloc[i]) else False
        if condition == 'or':
            if isin_kor | isin_eng:
                gold_ixs.append(df.iloc[[i], :].index[0])
        elif condition == 'and':
            if isin_kor & isin_eng:
                gold_ixs.append(df.iloc[[i], :].index[0])
        else:
            raise AssertionError("Unexpected condition. You can choose 'and' or 'or'. Default is 'or'.")
    gold_ixs = list(set(gold_ixs))
    return df.loc[gold_ixs, :].sort_index()

In [3]:
os.listdir('../')

['1장_20용어매핑테이블(기본진료료)_(심평원코드_SNOMED_CT).xlsx',
 '2장_19_20용어매핑테이블(검사)_(심평원코드-SNOMED_CT).xlsx',
 '3장_20용어매핑테이블(영상진단및방사선)_(심평원코드_SNOMED_CT).xlsx',
 '7장_20용어매핑테이블(이학요법료)_(심평원코드_SNOMED_CT).xlsx',
 '9장_19_20용어매핑테이블(처치및수술료)_(심평원코드-SNOMED_CT).xlsx',
 'code',
 'raw']

In [4]:
df = pd.read_excel('../9장_19_20용어매핑테이블(처치및수술료)_(심평원코드-SNOMED_CT).xlsx')

# 23.03.02 - coordinated concept; pre-, post- 를 통합. pre-가 없으면 post-에 있기 때문. 이름을 형용접두사 다 뗄거임.
for i in range(len(df)):
    if pd.isnull(df.iloc[i, 6]):
        df.iloc[i, 6] = df.iloc[i, 9]
        df.iloc[i, 7] = df.iloc[i, 10]
        df.iloc[i, 8] = df.iloc[i, 11]
df = df[np.setdiff1d(df.columns, df.columns[9:12], assume_unique=True)]
df = df[np.setdiff1d(df.columns, df.columns[3:5], assume_unique=True)]

df.rename(columns={
    '수가코드': 'feecode',
    '한글명': 'kor',
    '영문명': 'eng',
    'Precoordinated concept SCTID': 'Coordinated concept SCTID',
    'Precoordinated concept FSN': 'Coordinated concept FSN',
    'Precoordinated concept Hierarchy': 'Coordinated concept Hierarchy'
}, inplace=True)

list_korname = df['kor'].unique().tolist()
len(list_korname)

2598

In [5]:
df.columns

Index(['feecode', 'kor', 'eng', 'Mapping type', 'Coordinated concept SCTID',
       'Coordinated concept FSN', 'Coordinated concept Hierarchy',
       'Attribute SCTID', 'Attribute FSN', 'Attribute Hierarchy',
       'Value SCTID', 'Value FSN', 'Value Hierarchy', 'SNOMED CT Expression'],
      dtype='object')

In [6]:
for e in list_korname:
    if ("종양" in e) | ("암" in e):
        print(e)

혈관색전술-뇌혈관[척추포함]-종양-뇌에 화학요법제를 주입한 경우
혈관색전술-뇌혈관[척추포함]-종양-두경부에 화학요법제를 주입한 경우
혈관색전술-뇌혈관[척추포함]-종양-뇌
혈관색전술-뇌혈관[척추포함]-종양-두경부
혈관색전술-뇌혈관[척추포함]-종양-척추
경피적 간암 냉동제거술 [유도료 별도 산정]
경피적 냉동제거술 [유도료 별도 산정]-근골격계 종양
비촉지유방종양침위치결정술[유도료별도산정]
경피적폐암냉동제거술[유도료별도산정]
경피적 신장암 냉동제거술[유도료별도산정]
경피적 신장암의 고주파열치료술[유도료 별도산정]
폐암에 실시하는 경피적 고주파열치료술 [유도료 별도 산정]
경피적 고주파열치료술 [유도료 별도 산정]-부신종양
경피적 고주파열치료술 [유도료 별도 산정]-갑상선암
박피술[종양치료에 한함]-25㎠미만
박피술[종양치료에 한함]-25㎠이상
피부양성종양적출술(간단한표재성)
피부양성종양적출술(기타근육층에달하는것)
피부악성종양적출술(광역수술)
피부악성종양적출술(광역수술및림프절수술)
피부악성종양적출술(모즈미세도식수술)
연부조직종양적출술(지방종,혈관종,섬유종,거대세포종,화골성근염등)-악성종양
연부조직종양적출술(지방종,혈관종,섬유종,거대세포종,화골성근염등)-피하양성종양
연부조직종양적출술(지방종,혈관종,섬유종,거대세포종,화골성근염등)-근막하또는근육내양성종양
양성골종양의 소파술 또는 절제술(골반골,대퇴골,하퇴골)
양성골종양의 소파술 또는 절제술(상완골,전완골,쇄골)
양성골종양의 소파술 또는 절제술(기타)
악성골종양의 광범위절제술(골반골,대퇴골,하퇴골)
악성골종양의 광범위절제술(상완골,전완골,쇄골)
악성골종양의 광범위절제술(기타)
개두술 또는 두개절제술(두개골 병소 또는 양성종양절제)
개두술 또는 두개절제술(두개골 악성종양절제)
상악골(관골포함)양성종양(낭종포함)절제술-3cm미만
상악골(관골포함)양성종양(낭종포함)절제술-3cm이상
상악골(관골포함)악성종양[림프절청소포함]부분절제술
상악골(관골포함)악성종양[림프절청소포함]전적출술
하악골양성종양(낭종포함)절제술-편측악골1/3미만


In [7]:
# 23.02.27.
# 아래 키워드말고도 양성, 악성만 있고 암, 종양 단어가 없는 경우도 있다. 
# ex) "경피적간내홀뮴주입술[유도료별도산정]"
# 암, 종양, 림프, 양성, 악성 중 하나가 들어간 그룹 중 양성, 악성키워드만 존재하는 것이 있나. -> 있따

# 23.02.28.
# 적출도 키워드로 추가.

# 23.03.06.
# 아래 키워드가 들어가는 것을 df_cancer로 분류.
#       키워드 : 절단, 골암, 이식, 재건, 진단적, 절단, 수술
# 위에서 분류되지 않은 not_cancer에서 아래 코드에 해당하는 것은 cancer로 재편입.
#       코드 : Q2893, R4130, R4204

def tumor_keyword(u):
    return ("종양" in u) | ("암" in u) | ("림프" in u) | ("악성" in u) | \
        ("양성" in u) | ("적출" in u) | ("제거" in u) | ("절제" in u) | \
        ("이식" in u) | ("재건" in u) | ("절단" in u) | ("진단적" in u) | \
        ("수술" in u) | ("뇌기저" in u) | ("색전" in u)

df_cancer = pd.concat([df[df['kor']==u].iloc[[0], :] for u in df['kor'].unique() if tumor_keyword(u)], axis=0)
df_not_cancer = pd.concat([df[df['kor']==u].iloc[[0], :] for u in df['kor'].unique() if not tumor_keyword(u)], axis=0)


# df_etc2 = pd.concat([df_not_cancer.iloc[[i], :] for i in range(len(df_not_cancer)) if (df_not_cancer['feecode'].iloc[i] in ["Q2893", "R4130", "R4204"])], axis=0)
df_etc2 = goldendog(eng=['radical'], feecode=["Q2893", "R4130", "R4204"], condition='or', df=df_not_cancer)
df_cancer = pd.concat([df_cancer, df_etc2], axis=0).sort_index()


display(df_cancer.iloc[:, :2])
display(df_not_cancer.iloc[:, :2])
display(df_etc2.iloc[:, :2])
print("df_etc2 is in df_cancer.\n")

print("cancer + not_cancer = ", f"{len(df_cancer)} + {len(df_not_cancer)} = ", len(df_cancer) + len(df_not_cancer))

os.makedirs('./code9', exist_ok=True)
df_cancer.to_excel(f'./code9/{today}/cancer_{today}.xlsx')
df_not_cancer.to_excel(f'./code9/{today}/no_cancer_{today}.xlsx')


,feecode,kor
0,M0031,"피부 및 피하조직, 근육내 이물제거술[봉침, 파편 등]-근막절개하 이물제거술"
3,M0032,"피부 및 피하조직, 근육내 이물제거술[봉침, 파편 등]-기타"
11,M0074,직장분변제거술
32,M0134,수술후 튜브삽입에 의한 자연 배액감시 및 처치[1일당]
67,M0651,부정맥의 냉각절제술-냉각풍선절제술
...,...,...
3982,SZ711,골도보청기이식수술
3983,SZ712,청성뇌간이식
3999,T6131,"석고제거-견수상,고수상,체간캐스트"
4000,T6132,"석고제거-장상지,장하지,벨포캐스트"


,feecode,kor
4,M0040,산소흡입[1일당]
5,M0045,하기도증기흡입치료[1일당]
6,M0046,가온가습고유량비강캐뉼라요법
7,M0050,도뇨[1회당]
8,M0060,유치 카테터 설치
...,...,...
4011,T6154,부목-단하지[하퇴로부터 족부까지]
4012,T6155,부목-수지부
4013,T6161,"반창고부착술-흉곽부,하요추부"
4015,T6162,"반창고부착술-견관절부,고관절부"


,feecode,kor
594,N0550,흉벽냉농양근치술
2224,Q2893,.


df_etc2 is in df_cancer.

cancer + not_cancer =  1388 + 1212 =  2600


In [23]:
# 6 cancer in df_cancer
df_breast = goldendog(['유방'], ['breast', 'mast', '~localization'], df=df_cancer)
df_liver = goldendog(['간'], ['liver', 'hepa'], df=df_cancer)
df_colon = goldendog(['직장', '결장', '폴립'], ['colo', 'cole', 'sigmoid', 'rectal', 'mesent', '~vocal'], df=df_cancer)
df_gast = goldendog(['위', '상부소화관'], ['gast', 'muco'], df=df_cancer)
df_hyst = goldendog(['자궁'], ['hyst', '~vagi', 'uter', '~hydatid', '~correction', '~device'], df=df_cancer)
df_lung = goldendog(['폐', '기관지', '흉', '종격동', '횡격막'], ['lung', 'trach', 'bronc', 'wedge', 'lobec', 'segm', 'pneumo', 'chest', 'plur', 'ster', 'bullae', 'media', 'diaph', 'pulm'], df=df_cancer)

df_breast.to_excel(f'./code9/{today}/cBreast_{today}.xlsx')
df_liver.to_excel(f'./code9/{today}/cLiver_{today}.xlsx')
df_colon.to_excel(f'./code9/{today}/cColon_{today}.xlsx')
df_gast.to_excel(f'./code9/{today}/cGast_{today}.xlsx')
df_hyst.to_excel(f'./code9/{today}/cHyst_{today}.xlsx')
df_lung.to_excel(f'./code9/{today}/cLung_{today}.xlsx')

# ETC in df_cancer
target_ixs = df_breast.index.to_list() + df_liver.index.to_list() + df_colon.index.to_list() + df_gast.index.to_list() + df_hyst.index.to_list() + df_lung.index.to_list()
etc_ixs = np.setdiff1d(df_cancer.index, target_ixs)

df_etc = df_cancer.loc[etc_ixs]
df_etc.to_excel(f'./code9/cETC_{today}.xlsx')

print(f"BREAST\t\t+{len(df_breast):>4}")
print(f"COLON\t\t+{len(df_colon):>4}")
print(f"GAST\t\t+{len(df_gast):>4}")
print(f"HYST\t\t+{len(df_hyst):>4}")
print(f"LIVER\t\t+{len(df_liver):>4}")
print(f"LUNG\t\t+{len(df_lung):>4}")
print(f"ETC\t\t+{len(df_etc):>4}")
print("------------------------")
print(f"CANCERS\t\t {len(df_cancer):>4}\t+\tN.CANCER\t {len(df_not_cancer):>4}\t= TOTAL\t{len(df_cancer)+len(df_not_cancer):>4}")


BREAST		+  24
COLON		+  49
GAST		+  24
HYST		+  17
LIVER		+  49
LUNG		+  53
ETC		+1172
------------------------
CANCERS		 1388	+	N.CANCER	 1212	= TOTAL	2600


In [9]:
# Concatenate all dataframe
dfs = {
    'breast': df_breast,
    'colon': df_colon,
    'gast': df_gast,
    'hyst': df_hyst,
    'liver': df_liver,
    'lung': df_lung,
    'ETC': df_etc,
    'no_cancer': df_not_cancer
}

with pd.ExcelWriter(f'./code9/{today}/c6_{today}.xlsx') as exwr:
    for sheetname, that_df in dfs.items():
        that_df.to_excel(exwr, sheet_name=sheetname, index=False)
        ws = exwr.sheets[sheetname]
        for ix, col in enumerate(that_df):
            series = that_df[col]
            maxlen = max((series.astype(str).map(len).max(), len(str(series.name)))) + 1
            ws.set_column(ix, ix, maxlen)

    # df_breast.to_excel(exwr, sheet_name='breast', index=False)
    # df_colon.to_excel(exwr, sheet_name='colon', index=False)
    # df_gast.to_excel(exwr, sheet_name='gast', index=False)
    # df_hyst.to_excel(exwr, sheet_name='hyst', index=False)
    # df_liver.to_excel(exwr, sheet_name='liver', index=False)
    # df_lung.to_excel(exwr, sheet_name='lung', index=False)
    # df_etc.to_excel(exwr, sheet_name='ETC', index=False)
    # df_not_cancer.to_excel(exwr, sheet_name='no_cancer', index=False)

In [10]:
display(df_breast.iloc[:, :3])
display(df_liver.iloc[:, :3])
display(df_colon.iloc[:, :3])
display(df_gast.iloc[:, :3])
display(df_hyst.iloc[:, :3])
display(df_lung.iloc[:, :3])


,feecode,kor,eng
1128,N7121,유방양성종양절제술-단발성,Excision of Benign Breast Tumor-Single
1129,N7122,유방양성종양절제술-다발성,Excision of Benign Breast Tumor-Multiple
1130,N7131,유방절제술-양성-단순전절제,Mastectomy-Benign(Simple)
1131,N7132,유방절제술-양성-피하절제[남성의여성형유방절제포함],Mastectomy-Benign(Subcutaneous)
1133,N7133,유방절제술-양성-부분절제,Mastectomy-Benign(Partial)
1134,N7134,유방절제술-양성-액와부이소성유방절제,Mastectomy-Benign(Excision of Accessory Breast)
1135,N7136,유방절제술-악성-부분절제-액와림프절 청소술 포함,Mastectomy-Malignant(Partial)
1136,N7137,유방절제술-악성-부분절제-액와림프절 청소술 포함하지 않는 것,Mastectomy-Malignant(Partial)
1137,N7138,유방절제술-악성-전체절제-액와림프절 청소술 포함,Mastectomy-Malignant(Total)
1138,N7139,유방절제술-악성-전체절제-액와림프절 청소술 포함하지 않는 것,Mastectomy-Malignant(Total)


,feecode,kor,eng
278,M6774,경피적 간암 냉동제거술 [유도료 별도 산정],Percutaneous Cryosurgical Ablation of Liver Ca...
1952,Q0841,간암에 실시하는 경피적 열치료술 [유도료 별도 산정]-극초단파,Percutaneous Ablation of Liver Tumors-Microwave
2310,Q7211,간농양(낭종)수술-개복에 의한 흡인 또는 주입술,Operation of Liver Abscess(Cyst)-Aspiration or...
2314,Q7212,간농양(낭종)수술-배액을 위한 간절개술,Operation of Liver Abscess(Cyst)-Hepatotomy fo...
2316,Q7213,간농양(낭종)수술-조대술,Operation of Liver Abscess(Cyst)-Marsupialization
2319,Q7214,간농양(낭종)수술-낭종적출술,Operation of Liver Abscess(Cyst)-Excision of Cyst
2320,Q7221,간절제술-부분절제,Hepatectomy-Wedge Resection
2321,Q7222,간절제술-구역절제,Hepatectomy-Segmentectomy
2322,Q7223,간절제술-간엽절제,Hepatectomy-Lobectomy
2323,Q7224,간절제술-3구역절제,Hepatectomy-Trisegmentectomy


,feecode,kor,eng
1950,Q0292,직장및에스장절제술(초저위전방절제)-괄약근간절제술을 실시한 경우,Rectal and Sigmoid Resection-Ultra-Low anterio...
1956,Q1261,결장절제술(아전절제[우반및좌반결장동시절제또는좌반및에스상결장동시절제)-림프절청소포함,Colectomy-Subtotal
1957,Q1262,결장절제술(아전절제[우반및좌반결장동시절제또는좌반및에스상결장동시절제)-림프절청소포함하...,Colectomy-Subtotal
2136,Q2645,소장또는결장폴립절제술,Polypectomy of Small Bowel or Colon
2152,Q2671,결장절제술(우반 또는 좌반)-림프절청소를 포함하지 않는 것,Right or Left Hemicolectomy
2154,Q2672,결장절제술(전체)-림프절청소를 포함하지 않는 것,Colectomy-Total
2155,Q2673,결장절제술(부분절제)-림프절청소를 포함하지 않는 것,Colectomy-Segmental Resection
2156,Q2676,선천성거대결장수술(협착부근절제),Operation for Congenital Megacolon(Myomectomy)
2157,Q2679,"결장절제술 및 결장루,원위장 폐쇄[하트만수술]-림프절청소를 포함하지 않는 것",Colectomy With Proximal Colostomy And Distal S...
2159,Q2687,선천성거대결장근본수술(부분결장형),Operation for Congenital Megacolon(Radical)-Se...


,feecode,kor,eng
1936,Q0251,위아전절제술(부분절제)-림프절 청소를 포함하지 않는 것,Subtotal Gastrectomy(Partial)
1937,Q0252,위아전절제술(원위부절제)-림프절 청소를 포함하는 것,Subtotal Gastrectomy(Distal)
1939,Q0253,위아전절제술(원위부절제)-림프절 청소를 포함하지 않는 것,Subtotal Gastrectomy(Distal)
1940,Q0254,위아전절제술(유문부보존)-림프절 청소를 포함하는 것,Subtotal Gastrectomy(Pylous Preserving)
1942,Q0255,위아전절제술(유문부보존)-림프절 청소를 포함하지 않는 것,Subtotal Gastrectomy(Pylous Preserving)
1943,Q0256,위아전절제술(설상절제)-림프절 청소를 포함하는 것,Subtotal Gastrectomy(Wedge Resection)
1945,Q0257,위아전절제술(설상절제)-림프절 청소를 포함하지 않는 것,Subtotal Gastrectomy(Wedge Resection)
1946,Q0258,위아전절제술(근위부절제)-림프절 청소를 포함하는 것,Subtotal Gastrectomy(Proximal Resection)
1948,Q0259,위아전절제술-장관간치술 동시 실시한 경우,Subtotal Gastrectomy
2097,Q2510,"위절개술[이물제거,혈관결찰등 단순한 수술 포함]",Gastrotomy


,feecode,kor,eng
2641,R0141,전자궁적출술(림프절절제를 하지 않는 경우)-복강경술-단순,Hysterectomy(without Lymphadenectomy)-Laparosc...
2642,R0142,전자궁적출술(림프절절제를 하지 않는 경우)-복강경술-복잡[유착박리를 동반한 경우 또...,Hysterectomy(without Lymphadenectomy)-Laparosc...
2994,R4125,자궁경하자궁근종절제술-3cm미만,Hysteroscopic Removal of Leiomyoma
2995,R4126,자궁경하자궁근종절제술-3cm이상[다발성포함],Hysteroscopic Removal of Leiomyoma
2999,R4130,자궁질상부절단술,Subtotal Hysterectomy
3002,R4143,전자궁적출술(림프절절제를하는경우)-단순,Hysterectomy(With Lymphadenectomy)-Simple
3004,R4144,전자궁적출술(림프절절제를하는경우)-복잡[유착박리를동반한경우],Hysterectomy(With Lymphadenectomy)-Complex
3007,R4147,전자궁적출술(림프절절제를 하지 않는 경우)-복부접근-단순,Hysterectomy(without Lymphadenectomy)-Abdomina...
3008,R4148,전자궁적출술(림프절절제를 하지 않는 경우)-복부접근-복잡[유착박리를 동반한 경우],Hysterectomy(without Lymphadenectomy)-Abdomina...
3011,R4154,광범위자궁적출및양측골반림프절절제술-대동맥주위림프절생검을하는경우,Radical Hysterectomy With Bilateral Pelvic Lym...


,feecode,kor,eng
292,M6870,경피적폐암냉동제거술[유도료별도산정],Percutaneous Cryosurgical Ablation of Lung Cancer
295,M6900,폐암에 실시하는 경피적 고주파열치료술 [유도료 별도 산정],Percutaneous Radiofrequency Ablation of Lung C...
1441,O1311,기관또는기관지종양제거술(경부접근)-기관절제및재건술,Segmental Resection And End-To-End Anastomosis...
1443,O1312,기관또는기관지종양제거술(경부접근)-기관절제및성형술,"Segmental Resection And Tracheoplasty,Cervical..."
1445,O1313,기관또는기관지종양제거술(흉부접근)-기관(지)절제및재건술,Segmental Resection And End-To-End Anastomosis...
1447,O1314,기관또는기관지종양제거술(흉부접근)-기관(지)절제및성형술,Segmental Resection And Tracheoplasty(Bronchop...
1450,O1315,내시경적기관또는기관지종양제거술[육아조직포함]-연성기관지경,Flexible Bronchoscopic Excision of Tracheal or...
1451,O1316,내시경적기관또는기관지종양제거술[육아조직포함]-경성기관지경,Rigid Bronchoscopic Excision of Tracheal or Br...
1461,O1332,내시경적기관또는기관지이물제거술(연성기관지경),Flexible Bronchoscopic Removal of Tracheal or ...
1462,O1333,내시경적기관또는기관지이물제거술(경성기관지경),Rigid Bronchoscopic Removal of Tracheal or Bro...


In [11]:
[e for e in df['kor'] if "뇌" in e]

['혈관색전술-뇌혈관[척추포함]-동맥류-보조물지지',
 '혈관색전술-뇌혈관[척추포함]-동맥류-기타의경우',
 '혈관색전술-뇌혈관[척추포함]-동정맥기형[동정맥루포함]-뇌혈관',
 '혈관색전술-뇌혈관[척추포함]-동정맥기형[동정맥루포함]-뇌경막동정맥루(동맥경유)',
 '혈관색전술-뇌혈관[척추포함]-동정맥기형[동정맥루포함]-뇌경막동정맥루(정맥경유)',
 '혈관색전술-뇌혈관[척추포함]-동정맥기형[동정맥루포함]-내경동맥해면동루',
 '혈관색전술-뇌혈관[척추포함]-동정맥기형[동정맥루포함]-두경부',
 '혈관색전술-뇌혈관[척추포함]-동정맥기형[동정맥루포함]-척추',
 '혈관색전술-뇌혈관[척추포함]-동정맥기형[동정맥루포함]-척추경막',
 '혈관색전술-뇌혈관[척추포함]-종양-뇌에 화학요법제를 주입한 경우',
 '혈관색전술-뇌혈관[척추포함]-종양-뇌에 화학요법제를 주입한 경우',
 '혈관색전술-뇌혈관[척추포함]-종양-두경부에 화학요법제를 주입한 경우',
 '혈관색전술-뇌혈관[척추포함]-종양-두경부에 화학요법제를 주입한 경우',
 '혈관색전술-뇌혈관[척추포함]-종양-두경부에 화학요법제를 주입한 경우',
 '혈관색전술-뇌혈관[척추포함]-종양-두경부에 화학요법제를 주입한 경우',
 '혈관색전술-뇌혈관[척추포함]-종양-뇌',
 '혈관색전술-뇌혈관[척추포함]-종양-두경부',
 '혈관색전술-뇌혈관[척추포함]-종양-두경부',
 '혈관색전술-뇌혈관[척추포함]-종양-척추',
 '경피적풍선혈관성형술-뇌혈관',
 '경피적뇌혈관약물성형술',
 '경피적혈관내금속스텐트삽입술-뇌혈관',
 '천두술(낭종,혈종,농양제거 및 배액)-뇌실질내',
 '천두술(낭종,혈종,농양제거 및 배액)-뇌실질내',
 '천두술(낭종,혈종,농양제거 및 배액)-뇌실질내',
 '천두술(낭종,혈종,농양제거 및 배액)-뇌실질내',
 '지속적국소뇌혈류량측정 probe삽입술',
 '두개골성형술-경뇌막성형을동반하는것',
 '두개골성형술-경뇌막성형을동반하는것',
 '간적출술[이식용]-뇌사자(전간)',
 '간적출술[이식용]-뇌사자(전간)-우삼구역'

In [71]:
# 2023.03.07.
# 각 암별로 수술 부위 분류; 절제 정도와 절제 부위가 혼재되어 있어 두 개 칼럼으로 쪼개야 할 듯.
# surLv, surSite

def keyword_mes(df: pd.DataFrame, 
                new_column_placement: int=3, 
                new_colname: str="newcol",
                DUlikeNan: bool=False,
                **kwargs: list
                ) -> pd.DataFrame:
    ''' This can add easily some column's summary.

        Args:
            df: pd.DataFrame -> Input dataframe.
            new_columns_placement: int -> The new column is added at this number of column in the input dataframe. Default is 3.
            new_colname: str -> The new column's name.
            **kwargs: list -> Indexes of dataframe which you want to fill in column's summary. Just give like this `simple=[0, 1, 3, ...]`. Then it will add new column that locates in [0, 1, 3, ...], and has "simple" whether to be in list-index or "-".
        Returns:
            pd.DataFrame -> Output dataframe. It is added new column.
            
    '''
    assert isinstance(df, pd.DataFrame), "Input dataframe is expected as pd.DataFrame."

    new_copy_df = df.copy()
    blank_value = np.nan if DUlikeNan else "-"
    new_col = []

    for ix in new_copy_df.index:
        changed_token = False
        for key, value in kwargs.items():
            if ix in value:
                new_col.append(key)
                changed_token = True
                break
        if not changed_token:
            new_col.append(blank_value)
    new_copy_df.insert(new_column_placement, new_colname, new_col)

    return new_copy_df

def surgery_site_scrapper(target: str, *args: str):
    site_weaver = ""
    for arg in args:
        if arg in target:
            site_weaver += f"{arg},"
    if len(site_weaver) > 0:
        if site_weaver[-1] == ',':
            site_weaver = site_weaver[:-1]
    else:
        site_weaver = "-"
    return site_weaver

def filler(func, df, new_column_placement=3, new_colname="newcol"):
    def wrapper():
        func()
    return wrapper

# breast; lv
ix_breast_lv_simple = goldendog(['단순'], ['simple'], condition='or', df=df_breast).index
ix_breast_lv_sub = goldendog(['피하'], ['subcutaneous'], condition='or', df=df_breast).index
ix_breast_lv_partial = goldendog(['부분'], ['partial'], condition='or', df=df_breast).index
ix_breast_lv_total = goldendog(['전체'], ['total'], condition='or', df=df_breast).index
ix_breast_lv_recon = goldendog(['재건'], ['reconstruction'], df=df_breast).index
# breast; site
ix_breast_s_breast = goldendog(['유방', '~유두', '~유륜'], ['breast', 'mast', '~nipple', '~areolar'], condition='and', df=df_breast).index
ix_breast_s_bna = goldendog(['유방', '유두', '유륜'], ['breast', 'nipple', 'areolar'], condition='and', df=df_breast).index


# colon; lv
ix_colon_ultraLowAnterior = goldendog(['초저위'], ['ultra', 'anterior'], condition='or', df=df_colon).index
ix_colon_subtotal = goldendog(['아전'], ['subtotal'], condition='or', df=df_colon).index
ix_colon_total = goldendog(['전체'], ['total'], condition='or', df=df_colon).index
ix_colon_segmental = goldendog(['부분절제'], ['segmental'], condition='or', df=df_colon).index
ix_colon_myomec = goldendog(['협착부근'], ['myomec', 'congenital'], condition='or', df=df_colon).index
ix_colon_partialConmega = goldendog(['부분', '결장'], ['radical', 'segmental', 'congenital'], condition='or', df=df_colon).index
ix_colon_totalConmega = goldendog(['전', '결장'], ['radical', 'total', 'congenital'], condition='or', df=df_colon).index
ix_colon_transsacralORparasacral = goldendog(['경천골', '방천골'], ['transsacral', 'parasarcral'], condition='or', df=df_colon).index
ix_colon_transanal = goldendog(['경항문'], ['transanal'], condition='or', df=df_colon).index
# colon; site
ix_colon_s_bna = goldendog(['직장'], ['breast', 'nipple', 'areolar'], condition='and', df=df_breast).index



df_breast2 = keyword_mes(
    df=df_breast,
    simple=ix_breast_lv_simple,
    subcutaneous=ix_breast_lv_sub,
    partial=ix_breast_lv_partial,
    total=ix_breast_lv_total,
    reconstruction=ix_breast_lv_recon,
    new_colname="surLv"
    )


df_breast2 = keyword_mes(
    df=df_breast2,
    breast=ix_breast_s_breast,
    breast_nipple_areolar=ix_breast_s_bna,
    new_colname="surSite"
)

# df_colon2 = keyword_mes(
#     df=df_colon,
#     ultra_low_anterior=ix_colon_ultraLowAnterior,
#     subtotal=ix_colon_subtotal,
#     total=ix_colon_total,
#     myomec=ix_colon_myomec,
#     partial_congenital_megacolon=ix_colon_partialConmega,
#     total_congenital_megacolon=ix_colon_totalConmega,
#     transsacral_parasacral=ix_colon_transsacralORparasacral,
#     transanal=ix_colon_transanal
# )
        

In [79]:
df_breast2.iloc[:, :5]

,feecode,kor,eng,surSite,surLv
1128,N7121,유방양성종양절제술-단발성,Excision of Benign Breast Tumor-Single,breast,-
1129,N7122,유방양성종양절제술-다발성,Excision of Benign Breast Tumor-Multiple,breast,-
1130,N7131,유방절제술-양성-단순전절제,Mastectomy-Benign(Simple),breast,simple
1131,N7132,유방절제술-양성-피하절제[남성의여성형유방절제포함],Mastectomy-Benign(Subcutaneous),breast,subcutaneous
1133,N7133,유방절제술-양성-부분절제,Mastectomy-Benign(Partial),breast,partial
1134,N7134,유방절제술-양성-액와부이소성유방절제,Mastectomy-Benign(Excision of Accessory Breast),breast,-
1135,N7136,유방절제술-악성-부분절제-액와림프절 청소술 포함,Mastectomy-Malignant(Partial),breast,partial
1136,N7137,유방절제술-악성-부분절제-액와림프절 청소술 포함하지 않는 것,Mastectomy-Malignant(Partial),breast,partial
1137,N7138,유방절제술-악성-전체절제-액와림프절 청소술 포함,Mastectomy-Malignant(Total),breast,total
1138,N7139,유방절제술-악성-전체절제-액와림프절 청소술 포함하지 않는 것,Mastectomy-Malignant(Total),breast,total


In [78]:
def filler(func):
    def wrapper():
        func()
    return wrapper

@filler
def surgery_site_scrapper(target: str, *args: str):
    site_weaver = ""
    for arg in args:
        if arg in target:
            site_weaver += f"{arg},"
    if len(site_weaver) > 0:
        if site_weaver[-1] == ',':
            site_weaver = site_weaver[:-1]
    else:
        site_weaver = "-"
    return site_weaver

surgery_site_scrapper("abcef", 'a', 'd')

# , df, new_column_placement=3, new_colname="newcol"

TypeError: wrapper() takes 0 positional arguments but 3 were given